1. 대구의 7개 구(중구, 동구, 서구, 남구, 북구, 수성구, 달서구)과 1개 군(달성군)의 맛집은 각각 몇 개인가?

2. 오픈 api에 포함된 "설명"(SMPL_DESC)을 분석하여, 맛집 설명에서 가장 유의미하게 많이 등장하는 5개 단어를 제시하라. 

3. 오픈 api에 포함된 "메뉴"(MNU)를 분석하여, 대구 맛집들이 제공하는 가장 흔한 메뉴 3개를 제시하라. 그리고 이들의 평균 가격 또한 함께 제시하라.

4. 본인이 제시한 2번, 3번 결과에 부합하는 맛집을 최종적으로 최대 3곳 선정하여 제시하라. (논리적 근거를 명확히 밝히라.)

 

* 답변의 제출은 반드시 "학번.pdf"로 제출하고, 추가적으로 분석에 활용된 python code를 원본형식으로 제출하라.

(코드 파일에 # 이나 Jupyternote book 기능을 이용해 설명만 추가하는 형식의 제출은 정식 과제 점수로 인정하지 않음.)

In [1]:
import requests
from urllib import parse 

In [2]:
def matzip_api(coordinate):
    address = 'https://www.daegufood.go.kr/kor/api/tasty.html?mode=json&addr='
    coordinate = parse.quote(coordinate)
    response = requests.get(address+coordinate)
    
    return response.json(strict=False)

In [3]:
location = ['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군']

In [4]:
daegu_matzip = {i:matzip_api(i) for i in location}

### 1. 대구의 7개 구(중구, 동구, 서구, 남구, 북구, 수성구, 달서구)과 1개 군(달성군)의 맛집은 각각 몇 개인가?

In [5]:
for i in daegu_matzip:
    print(f'{i}: {daegu_matzip[i]["total"]} 개')

중구: 181 개
동구: 137 개
서구: 63 개
남구: 57 개
북구: 95 개
수성구: 131 개
달서구: 168 개
달성군: 93 개


### 2. 오픈 api에 포함된 "설명"(SMPL_DESC)을 분석하여, 맛집 설명에서 가장 유의미하게 많이 등장하는 5개 단어를 제시하라.

In [6]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

In [7]:
def make_df(location):
    df = []
    for i in location:
        local_df = pd.DataFrame(daegu_matzip[i]['data'])
        local_df['LOCATION'] = i
        df.append(local_df)
        
    return pd.concat(df).reset_index(drop=True)

In [8]:
df = make_df(location)

In [9]:
df['FD_CS'].value_counts()

한식           700
양식            41
일식            40
디저트/베이커리      38
중식            36
세계요리          34
전통차/커피전문점     19
퓨전/뷔페         11
특별한 술집         6
Name: FD_CS, dtype: int64

In [10]:
clean = re.compile("[^ㄱ-힣0-9 %]")
df['text'] = df['SMPL_DESC'].apply(lambda x: clean.sub('', str(x)))

In [11]:
def tokenize(text):
    okt = Okt()
    word_token = okt.nouns(text)
    result = [word for word in word_token if len(word) > 1]
    return word_token

In [15]:
texts = df['text'].apply(tokenize)

In [16]:
texts

0                          [송림, 복어, 이상, 전통, 복어, 요리, 전문점]
1      [테이, 스팅, 스푼, 은, 가격, 스테이크, 파스타, 수, 가성, 비, 양식, 전문점]
2      [이상, 자리, 온, 동성로, 명물, 박석, 규, 중앙, 떡볶이, 쌀, 떡볶이, 납...
3             [핸드, 메이드, 모든, 음식, 직접, 가성, 비, 유러, 피안, 레스토랑]
4      [차이, 미들급, 다이닝, 지향, 태국, 요리, 및, 일본, 요리, 등, 아시안, ...
                             ...                        
920    [대형, 양어, 지하, 수로, 정수, 장어, 건강, 잔디밭, 전망, 탁트인, 야외,...
921    [계곡, 물, 소리, 바람소리, 방갈로, 전통, 염소, 보양, 요리, 곳, 주인, ...
922    [뫼, 카페, 산, 배경, 경관, 입구, 통나무, 입, 판이, 인상, 분위기, 추천...
923    [대자연, 식당, 도심, 수, 새, 노래, 소리, 계곡, 물, 소리, 바람소리, 신...
924    [가창, 위치, 파스타, 피자, 전문점, 송하, 정원, 산책, 엔틱, 유럽, 층, ...
Name: text, Length: 925, dtype: object

In [17]:
from gensim import corpora

dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(text) for text in texts]

In [26]:
import gensim

NUM_TOPICS = 1
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics()

In [28]:
df_foreign = df[df['FD_CS'] == '양식']['SMPL_DESC'].apply(lambda x: clean.sub('', str(x)))

In [33]:
texts = df_foreign.apply(tokenize)
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
NUM_TOPICS = 1
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics()

In [34]:
topics

[(0,
  '0.024*"수" + 0.020*"레스토랑" + 0.019*"전문점" + 0.018*"파스타" + 0.016*"피자" + 0.016*"스테이크" + 0.013*"요리" + 0.011*"위치" + 0.010*"곳" + 0.009*"음식점"')]